In [34]:
import os
import json
from pprint import pprint

from ase.visualize import view
from pymatgen.ext.matproj import MPRester
from pymatgen.io.cif import CifWriter
from pymatgen.io.ase import AseAtomsAdaptor

from aiida import load_profile
from aiida.orm import Float, Int, Str, Bool, Code, Dict, StructureData, KpointsData, UpfData, load_node
from aiida.orm.nodes.data.structure import StructureData
from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain
from aiida_quantumespresso.workflows.pw.relax import PwRelaxWorkChain
from aiida_quantumespresso.workflows.pw.bands import PwBandsWorkChain
from aiida.engine.processes.builder import ProcessBuilder
from aiida.engine import submit

In [24]:
m = MPRester(os.getenv('MP_API_KEY'))
load_profile()


Profile<uuid='4fe00b7a32994cfca8b6cf43c2d22a53' name='jgeiger'>

In [32]:
si_bulk_pmg = m.get_structure_by_material_id('mp-149')

si_bulk_structuredata = StructureData(pymatgen_structure=si_bulk_pmg)
si_bulk_ase = AseAtomsAdaptor.get_atoms(si_bulk_pmg)

si_bands_builder = PwBandsWorkChain.get_builder_from_protocol(
    code='qe-dev-pw',
    structure=si_bulk_structuredata,
)

si_builder_dict = si_bands_builder.items()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
# si_builder._repr_pretty_()

In [33]:
# si_bulk_base_node = submit(si_test_builder)
# view(si_bulk_ase)
for k,v in si_bands_builder.items():
    print(k, v)

metadata {}
relax {'metadata': {}, 'base': {'metadata': {}, 'pw': {'metadata': {'options': {'stash': {}, 'resources': {'num_machines': 1}, 'max_wallclock_seconds': 43200, 'withmpi': True}}, 'monitors': {}, 'pseudos': {'Si': <UpfData: uuid: 23019765-e357-4fc8-b394-472defb13ace (pk: 20)>}, 'code': <InstalledCode: Remote code 'qe-dev-pw' on lumi-small pk: 2182, uuid: b6e74e35-9fb2-45d2-8872-a430f85e44b3>, 'parameters': <Dict: uuid: 292a4045-5e97-401f-b542-69f06bb37e51 (unstored)>}, 'kpoints_distance': <Float: uuid: 1912be6b-6a0e-409e-9586-c80e23b6e32f (unstored) value: 0.15>, 'kpoints_force_parity': <Bool: uuid: 36bbf1f3-c486-4f6f-a384-13a9fa130b88 (unstored) value: False>}, 'base_final_scf': {'metadata': {}, 'pw': {'metadata': {'options': {'stash': {}}}, 'monitors': {}, 'pseudos': {}}}, 'max_meta_convergence_iterations': <Int: uuid: f5f3b646-7f0f-4aa0-9673-da1577a50a0e (unstored) value: 5>, 'meta_convergence': <Bool: uuid: 82a2cf09-7ce2-4715-92d4-02309cc64302 (unstored) value: True>, 'vo

In [10]:

view(si_bulk_ase)

In [35]:
si_bands_node = submit(si_bands_builder)

In [2]:
%%bash
verdi process status 2364
verdi process report 2364

PwBandsWorkChain<2364> Finished [0] [7:results]
    ├── PwRelaxWorkChain<2366> Finished [0] [3:results]
    │   ├── PwBaseWorkChain<2369> Finished [0] [4:results]
    │   │   ├── create_kpoints_from_distance<2370> Finished [0]
    │   │   └── PwCalculation<2374> Finished [0]
    │   └── PwBaseWorkChain<2383> Finished [0] [4:results]
    │       ├── create_kpoints_from_distance<2384> Finished [0]
    │       └── PwCalculation<2388> Finished [0]
    ├── seekpath_structure_analysis<2395> Finished [0]
    ├── PwBaseWorkChain<2402> Finished [0] [4:results]
    │   ├── create_kpoints_from_distance<2403> Finished [0]
    │   └── PwCalculation<2407> Finished [0]
    └── PwBaseWorkChain<2415> Finished [0] [4:results]
        └── PwCalculation<2418> Finished [0]
2023-02-20 11:07:39 [84  | REPORT]: [2364|PwBandsWorkChain|run_relax]: launching PwRelaxWorkChain<2366>
2023-02-20 11:07:40 [85  | REPORT]:   [2366|PwRelaxWorkChain|run_relax]: launching PwBaseWorkChain<2369>
2023-02-20 11:07:43 [86  | R

In [9]:
%%bash
verdi process show 2364

Property     Value
-----------  ------------------------------------
type         PwBandsWorkChain
state        Finished [0]
pk           2364
uuid         0a88bce9-a68e-4243-9d97-22724dc7b3cd
label
description
ctime        2023-02-20 11:07:35.134100+01:00
mtime        2023-02-20 11:17:38.724354+01:00

Inputs                               PK    Type
-----------------------------------  ----  -------------
bands
    pw
        pseudos
            Si                       20    UpfData
        code                         2182  InstalledCode
        parameters                   2358  Dict
    max_iterations                   2359  Int
relax
    base
        pw
            pseudos
                Si                   20    UpfData
            code                     2182  InstalledCode
            parameters               2347  Dict
        kpoints_distance             2348  Float
        kpoints_force_parity         2349  Bool
        max_iterations               2350  Int
    max_meta_

In [10]:
%%bash
verdi data core.bands export --help
verdi data core.bands export  -v debug --format mpl_pdf --output si_band_structure.pdf 2421

Usage: verdi data core.bands export [OPTIONS] DATUM

  Export BandsData objects.

Options:
  -F, --format [agr|agr_batch|dat_blocks|dat_multicolumn|gnuplot|json|mpl_pdf|mpl_png|mpl_singlefile|mpl_withjson]
                                  Format of the exported file.  [default:
                                  json]
  --y-min-lim FLOAT               The minimum value for the y axis. Default:
                                  minimum of all bands
  --y-max-lim FLOAT               The maximum value for the y axis. Default:
                                  maximum of all bands
  -o, --output TEXT               If present, store the output directly on a
                                  file with the given name. It is essential to
                                  use this option if more than one file needs
                                  to be created.
  -f, --force                     If passed, overwrite files without checking.
  --prettify-format [agr_seekpath|agr_simple|gnuplot_s